# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database

print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# query documents with a scores Hygiene rating equal to 20
query = {"scores.Hygiene": 20}
fields = {"_id": 0, "BusinessName": 1, "scores.Hygiene": 1}
results = establishments.find(query, fields)

In [7]:
# counn_documents() method to count the number of documents that match the query
print(establishments.count_documents(query))

41


In [8]:
# set the options before executing the query
query = {"scores.Hygiene": 20}
fields = {"_id": 0, "BusinessName": 1, "scores.Hygiene": 1}

# execute the query
results = establishments.find(query, fields)

# print the first result
pprint(results[0])

{'BusinessName': 'The Chase Rest Home', 'scores': {'Hygiene': 20}}


In [9]:
# create a dataframe from the results
df = pd.DataFrame(results)

In [10]:
# display the first 10 rows
df.head(10)


,BusinessName,scores
0,The Chase Rest Home,{'Hygiene': 20}
1,Brenalwood,{'Hygiene': 20}
2,Melrose Hotel,{'Hygiene': 20}
3,Seaford Pizza,{'Hygiene': 20}
4,Golden Palace,{'Hygiene': 20}
5,Ashby's Butchers,{'Hygiene': 20}
6,South Sea Express Cuisine,{'Hygiene': 20}
7,Golden Palace,{'Hygiene': 20}
8,The Tulip Tree,{'Hygiene': 20}
9,F & S,{'Hygiene': 20}


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [37]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4 using Regex to find the establishments with London as the Local Authority
rating_value_query = {'LocalAuthorityName': {'$regex': 'London', '$options': 'i'}, 'RatingValue': {'$gte': 4}}
fields = {'_id': 0, 'BusinessName': 1, 'RatingValue': 1, 'LocalAuthorityName': 1}
rating_value_result = establishments.find(rating_value_query, fields)

In [35]:
# Use count_documents to display the number of documents in the result
print(establishments.count_documents(rating_value_query))

33


In [38]:
# print the first result
pprint(rating_value_result[0])

{'BusinessName': "Charlie's",
 'LocalAuthorityName': 'City of London Corporation',
 'RatingValue': 4}


In [39]:
# Convert the result to a Pandas DataFrame
rating_value_df = pd.DataFrame(rating_value_result)

In [40]:
# Display the number of rows in the DataFrame
print(rating_value_df.shape[0])

33


In [41]:
# Display the first 10 rows of the DataFrame
df.head(10)

,BusinessName,RatingValue,LocalAuthorityName
0,Charlie's,4,City of London Corporation
1,Mv City Cruises Erasmus,5,City of London Corporation
2,Benfleet Motor Yacht Club,4,City of London Corporation
3,Coombs Catering t/a The Lock and Key,5,City of London Corporation
4,Tilbury Seafarers Centre,5,City of London Corporation
5,Mv Valulla,5,City of London Corporation
6,Tereza Joanne,5,City of London Corporation
7,Brick Lane Brews,4,City of London Corporation
8,The Nuance Group (UK) Limited,5,City of London Corporation
9,WH Smith,5,City of London Corporation


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude =
longitude =

query =
sort =
limit =

# Print the results


SyntaxError: invalid syntax (3510804407.py, line 6)

In [ ]:
# Convert result to Pandas DataFrame


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
